In [1]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
print("pandas: %s" % pd.__version__)

import sklearn
from sklearn.preprocessing import StandardScaler
print("sklearn: %s" % sklearn.__version__)

pandas: 1.0.1
sklearn: 0.22.1


In [2]:
# Display all the columns for the dataframes (not-truncated)
pd.set_option("display.max_columns", None)

In [3]:
# Read `crypto_data.csv` into Pandas. 
df = pd.read_csv("crypto_data.csv")

print(df.describe())
df

       TotalCoinsMined
count     7.440000e+02
mean      3.883103e+09
std       3.867263e+10
min      -5.917978e+09
25%       6.322164e+05
50%       1.340427e+07
75%       9.399895e+07
max       9.899887e+11


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [4]:
def normalize_decimal_separator(line):
    """
    clean the string number from containing spaces 
    and underscores as the thousands seperator and
    also remove any commas or periods as the 
    thousands seperator
    
    returns a string
    """
    try:
        float(line)
        return line
    except ValueError: 
        # remove the spaces as the thousands seperator
        if (" " in line):
            line = line.replace(" ", "")
        # remove the underscore as the thousands seperator
        if ("_" in line):
            line = line.replace("_", "")
        # remove periods if they are the thousands seperator
        if (line.count(".") > 1):
            line = line.replace(".", "")
        # remove commas if they are the thousands seperator
        if (line.count(",") > 1):
            line = line.replace(",", "")
        #
        return line

In [5]:
# Fix the error in the total coin supply where the values are coming in as a string
if not is_numeric_dtype(df["TotalCoinSupply"]):
    df["TotalCoinSupply"] = pd.to_numeric(df["TotalCoinSupply"].apply(normalize_decimal_separator))
    print(df["TotalCoinSupply"].describe())
df

count    1.252000e+03
mean     8.728784e+14
std      2.837954e+16
min      0.000000e+00
25%      1.730442e+07
50%      5.412806e+07
75%      3.500000e+08
max      1.000000e+18
Name: TotalCoinSupply, dtype: float64


,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,4.200000e+01
1,365,365Coin,X11,True,PoW/PoS,NaN,2.300000e+09
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,5.320000e+08
3,611,SixEleven,SHA-256,True,PoW,NaN,6.110000e+05
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1.000000e+06
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,1.000000e+08
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,2.331000e+08
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,2.100000e+07


In [6]:
# Discard all cryptocurrencies that are not being traded
df = df[df["IsTrading"] == True]
df["IsTrading"].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [7]:
# Drop the `IsTrading` column from the dataframe
df = df.drop(columns=["IsTrading"])
df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
1,365,365Coin,X11,PoW/PoS,NaN,2.300000e+09
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
3,611,SixEleven,SHA-256,PoW,NaN,6.110000e+05
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,PoW,NaN,1.000000e+09
1244,UOS,UOS,SHA-256,DPoI,NaN,1.000000e+09
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07


In [8]:
# Remove all rows that have at least one null value
print(df.isnull().sum())
df = df.dropna()
df

Unnamed: 0           0
CoinName             0
Algorithm            0
ProofType            0
TotalCoinsMined    459
TotalCoinSupply      0
dtype: int64


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0.000000e+00
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07


In [9]:
# Filter for cryptocurrencies that have been mined
df = df[df["TotalCoinsMined"] > 0]
df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,3.141593e+11
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,2.100000e+07
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0.000000e+00
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2.000000e+09
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1.400223e+09
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,2.100000e+07


In [10]:
# delete the `Unnamed: 0` and `CoinName` from the original dataframe
if ("Unnamed: 0" in df.columns):
    df = df.drop(columns=["Unnamed: 0","CoinName"])
else:
    df = df.drop(columns=["CoinName"])
df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,4.200000e+01
2,Scrypt,PoW/PoS,1.055185e+09,5.320000e+08
5,X13,PoW/PoS,2.927942e+10,3.141593e+11
7,SHA-256,PoW,1.792718e+07,2.100000e+07
8,Ethash,PoW,1.076842e+08,0.000000e+00
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2.000000e+09
1242,Scrypt,PoW/PoS,1.493105e+07,2.500000e+08
1245,CryptoNight,PoW,9.802226e+08,1.400223e+09
1246,Equihash,PoW,7.296538e+06,2.100000e+07


In [11]:
df.describe()

,TotalCoinsMined,TotalCoinSupply
count,5.320000e+02,5.320000e+02
mean,5.340456e+09,1.034523e+10
std,4.564568e+10,6.773697e+10
min,4.199995e+01,0.000000e+00
25%,8.359849e+06,2.100000e+07
50%,2.474397e+07,7.841760e+07
75%,1.867250e+08,5.000000e+08
max,9.899887e+11,1.000000e+12


In [12]:
df.to_csv("Resources/crypto_clean_data.csv", index=False)